In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

from src.attacks import create_attack
from src.graph_models.csbm import CSBM
from src.models.common import get_diffusion
from src.models.ntk import NTK
from common import get_graph
from src import utils

In [3]:
# Data
data_dict = dict(
    classes = 2,
    n = 1000,
    n_per_class_trn = 300,
    n_per_class_labeled = 300,
    sigma = 1,
    avg_within_class_degree = 1.58 * 2,
    avg_between_class_degree = 0.37 * 2,
)
K = 1
seed = 0
# Model
model_dict = {
    "label": "GCN",
    "model": "GCN",
    "normalization": "row_normalization",
    "depth": 1,
}
attack_params = {
    "attack": "random"
}
# Attack
eps_l = [0, 0.01, 0.025, 0.05, 0.10, 0.25, 0.50, 1, 2.5, 5, 10]
# other
device = "0"
dtype = torch.float64

In [5]:
data_dict["K"] = K
X, A, y = get_graph(data_dict, seed=seed, sort=True)

In [8]:
X

array([[ 0.99598831, -1.15782773,  0.19027323, ...,  0.45553085,
        -0.56686521, -0.88342708],
       [ 0.06456569, -1.27180827,  1.43475187, ..., -0.08360106,
         0.94913799, -0.82104301],
       [-0.39911875,  1.15486872,  0.79823089, ..., -1.34563637,
         1.40735793, -0.15816568],
       ...,
       [ 0.20783108,  0.69879353,  0.74385655, ...,  1.3015219 ,
         1.33653069, -1.98341584],
       [-3.17073846,  0.66761136, -1.53758502, ...,  0.9402824 ,
         0.3452377 ,  2.63514113],
       [ 0.86973256, -0.13959238, -1.02761626, ..., -0.42092168,
         1.89871967,  2.08923745]])

In [5]:
rng = np.random.Generator(np.random.PCG64(seed))
ntk_dict = dict()
device_ = configure_hardware(device, seed)
data_dict["K"] = K
# Sample
X, A, y = get_graph(data_dict, seed=seed, sort=True)
X = torch.tensor(X, dtype=dtype, device=device_)
A = torch.tensor(A, dtype=dtype, device=device_)
y = torch.tensor(y, device=device_)
# Trn / Test Split
n_cls0 = sum(y == 0).cpu().item()
n = len(y)
idx_cls0 = rng.permutation(np.arange(n_cls0))
idx_cls1 = rng.permutation(np.arange(n_cls0, n))
n_trn = data_dict["n_per_class_trn"]
n_labeled = data_dict["n_per_class_labeled"]
idx_labeled = np.concatenate((idx_cls0[:n_labeled], idx_cls1[:n_labeled]))
idx_unlabeled = np.concatenate((idx_cls0[n_labeled:], idx_cls1[n_labeled:]))
idx_target = np.concatenate((idx_cls0[n_trn:], idx_cls1[n_trn:]))

# Computing NTK
ntk = NTK(model_dict, X, A)
attack = create_attack(idx_target, X, A, y, idx_labeled, idx_unlabeled, 
                       attack_params, seed)
# Computing NTK
ntk_l = list()
acc_l = list()
for eps in eps_l:
    n_pert = int(round(eps * count_edges_for_idx(A.cpu(), idx_target)))
    A_pert = attack.attack(n_pert).detach().clone()
    ntk_pert = NTK(model_dict, X, A_pert, device=device_)
    y_pred = ntk_pert(idx_labeled, idx_unlabeled, y)
    acc_l.append(utils.accuracy(y_pred, y[idx_target]).cpu().item())
    ntk_l.append(ntk_pert)

In [6]:
print(acc_l)

[0.7724999785423279, 0.7649999856948853, 0.7674999833106995, 0.7524999976158142, 0.7049999833106995, 0.6225000023841858, 0.5099999904632568, 0.4675000011920929, 0.4599999785423279, 0.7400000095367432, 0.8974999785423279]


In [23]:
eps = 0.05
n_pert = int(round(eps * count_edges_for_idx(A.cpu(), idx_target)))
A_pert = attack.attack(n_pert).detach().clone()
print((A_pert > 1).sum())

tensor(0., device='cuda:0', dtype=torch.float64)
tensor(1., device='cuda:0', dtype=torch.float64)
tensor([245, 131,  96, 158, 240, 268, 383, 156, 458, 390, 179, 175, 357,  34,
         22, 152, 161,  58, 477, 196,  79, 134, 172, 117, 460, 352, 156, 296,
        347, 317, 361, 122, 247, 263, 424, 411, 384, 206, 163, 317, 356, 377,
        340, 381, 107, 303, 315, 384, 105, 422, 226, 207, 490,  64, 216, 309,
         99, 106, 258, 426, 477], device='cuda:0')
tensor([740, 526, 661, 721, 786, 918, 710, 951, 657, 947, 643, 862, 526, 920,
        982, 509, 628, 546, 590, 713, 974, 963, 665, 516, 966, 693, 940, 772,
        751, 547, 815, 937, 623, 825, 930, 817, 873, 851, 775, 602, 731, 833,
        586, 728, 986, 878, 692, 585, 546, 517, 862, 747, 963, 725, 984, 527,
        759, 829, 974, 985, 813], device='cuda:0')
tensor(1., device='cuda:0', dtype=torch.float64)
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')


In [13]:
count_edges_for_idx(A, idx_target)

RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)